In [1]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor, pipeline

# === Настройки устройства ===
device = "cpu"
torch_dtype = torch.float32  # CPU обычно работает с float32
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"  # Apple GPU
    setattr(torch.distributed, "is_initialized", lambda: False)  # monkey patch

device = torch.device(device)

# === Загружаем модель и процессор ===
model_id = "antony66/whisper-large-v3-russian"

print("⏳ Загружаем модель Whisper Large V3 Russian...")
whisper = WhisperForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True,
)

processor = WhisperProcessor.from_pretrained(model_id)

# === Создаём пайплайн ASR ===
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=whisper,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=256,
    chunk_length_s=30,       # обрабатываем кусками по 30 сек
    batch_size=16,
    return_timestamps=True,  # для длинных файлов
    torch_dtype=torch_dtype,
    device=device,
)

# === Путь к файлу ===
audio_path = "/home/ppwwrr/project_25/voices/voice-2.wav"

# === Транскрибируем аудио ===
print(f"🎧 Обрабатываем файл: {audio_path}...")
asr = asr_pipeline(
    audio_path,
    generate_kwargs={"language": "russian", "max_new_tokens": 256},
    return_timestamps=False  # нам не нужны таймкоды
)

# === Выводим результат ===
print("\n✅ Транскрипция завершена:\n")
print(asr["text"])


`torch_dtype` is deprecated! Use `dtype` instead!


⏳ Загружаем модель Whisper Large V3 Russian...


generation_config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


🎧 Обрабатываем файл: /home/ppwwrr/project_25/voices/voice-2.wav...

✅ Транскрипция завершена:

 что последний раз я смотрела короче у меня вот эти духи появились чтобы не соврать десятом или в одиннадцатом классе вот типа вот либо конец десятая либо начало одиннадцатого ну короче вот они у меня появились мне мама заказала потому что это духи полиного отца а вот потому что пол Полина, они, унисекс, она их носила, мне так понравилось, я такая, блин, что это, я пришла к ней домой, сфоткала, и все, и мы их купили, еще тогда я смотрела, типа, на золотом яблоке они стоили, типа, ну, девятнадцать, ну, восемнадцать, ну, как бы, да,Да,, ну, нишевая парфюмерия, как бы, ладно, еще можно как бы схавать, но сорок пять, блядь.
